In [1]:
%%capture
!pip install torch_geometric rdkit
!pip install torch-scatter -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html

In [2]:
# Standard library
from collections import defaultdict
from typing import List, Tuple
import importlib.resources as pkg_resources
from multiprocessing.pool import ThreadPool

# Third-party scientific stack
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh

# RDKit
from rdkit import Chem

# PyTorch core
import torch
import torch.nn as nn
import torch.nn.functional as F

# PyTorch Geometric
import torch_geometric
from torch_geometric.data import Data, Batch, DataLoader
from torch_geometric.nn import GCNConv, GINConv, BatchNorm, global_mean_pool
from torch_geometric.loader import DataLoader as PyGDataLoader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def set_seed(seed):
    """
    Fix all random seeds for reproducibility across Python, NumPy, and PyTorch.
    """
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import (
    PNAConv,
    global_mean_pool,
    global_max_pool,
    global_add_pool
)
from torch_geometric.utils import degree

class PolyatomicNet(nn.Module):
    def __init__(self, node_feat_dim, edge_feat_dim, graph_feat_dim,deg,
                 hidden_dim=128, num_layers=5, dropout=0.1):
        super().__init__()

        self.node_emb = nn.Linear(node_feat_dim, hidden_dim)
        self.deg = deg
        self.virtualnode_emb = nn.Embedding(1, hidden_dim)
        self.vn_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # For graph-level feature projection
        self.graph_proj = nn.Sequential(
            nn.Linear(graph_feat_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # PNAConv requires degree preprocessing
        self.deg_emb = nn.Embedding(20, hidden_dim)  # cap degree buckets

        aggregators = ['mean', 'min', 'max', 'std']
        scalers = ['identity', 'amplification', 'attenuation']

        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()

        for _ in range(num_layers):
            conv = PNAConv(
                in_channels=hidden_dim,
                out_channels=hidden_dim,
                aggregators=aggregators,
                scalers=scalers,
                edge_dim=edge_feat_dim,
                towers=4,
                pre_layers=1,
                post_layers=1,
                divide_input=True,
                deg=deg
            )
            self.convs.append(conv)
            self.bns.append(nn.BatchNorm1d(hidden_dim))

        self.dropout = nn.Dropout(dropout)

        # Final readout
        self.readout = nn.Sequential(
            nn.Linear(hidden_dim * 3, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1),
        )

    def forward(self, data):
        x, edge_index, edge_attr, batch = (
            data.x,
            data.edge_index,
            data.edge_attr,
            data.batch,
        )

        deg = degree(edge_index[0], x.size(0), dtype=torch.long).clamp(max=19)
        h = self.node_emb(x) + self.deg_emb(deg)

        vn = self.virtualnode_emb(
            torch.zeros(batch.max().item() + 1, dtype=torch.long, device=x.device)
        )

        for conv, bn in zip(self.convs, self.bns):
            h = h + vn[batch]
            h = conv(h, edge_index, edge_attr)
            h = bn(h)
            h = F.relu(h)
            h = self.dropout(h)
            vn = vn + self.vn_mlp(global_mean_pool(h, batch))

        mean_pool = global_mean_pool(h, batch)
        max_pool  = global_max_pool(h, batch)
        # add_pool  = global_add_pool(h, batch)

        # graph_feats shape: [batch_size, graph_feat_dim]
        if hasattr(data, 'graph_feats') and isinstance(data, torch_geometric.data.Batch):
            g_feats = torch.stack([g.graph_feats for g in data.to_data_list()], dim=0).to(x.device)
        else:
            g_feats = data.graph_feats.unsqueeze(0).to(x.device)
        g_proj = self.graph_proj(g_feats)

        final_input = torch.cat([mean_pool, max_pool, g_proj], dim=1)
        return self.readout(final_input).view(-1)

In [6]:
from torch.amp import autocast, GradScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

scaler_grad = GradScaler()

def train(model, loader, optimizer, loss_fn, scaler=scaler_grad, accum_steps=8):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()

    for i, batch in enumerate(loader):
        batch = batch.to(device)
        with autocast(device_type='cuda', dtype=torch.float16):
            output = model(batch)
            loss = loss_fn(output, batch.y.view(-1)) / accum_steps

        scaler_grad.scale(loss).backward()

        if (i + 1) % accum_steps == 0 or (i + 1 == len(loader)):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item() * batch.num_graphs * accum_steps

    return total_loss / len(loader.dataset)

def compute_metrics_with_ci(trues, preds, n_boot=2000, alpha=0.05, seed=42):
    trues = np.array(trues)
    preds = np.array(preds)
    mae = mean_absolute_error(trues, preds)
    rmse = np.sqrt(mean_squared_error(trues, preds))

    rng = np.random.RandomState(seed)
    mae_samples, rmse_samples = [], []
    n = len(trues)
    for _ in range(n_boot):
        idx = rng.randint(0, n, n)
        t, p = trues[idx], preds[idx]
        mae_samples.append(mean_absolute_error(t, p))
        rmse_samples.append(np.sqrt(mean_squared_error(t, p)))

    lower, upper = 100 * alpha / 2, 100 * (1 - alpha / 2)
    mae_ci = (np.percentile(mae_samples, lower), np.percentile(mae_samples, upper))
    rmse_ci = (np.percentile(rmse_samples, lower), np.percentile(rmse_samples, upper))
    return {'mae': mae, 'mae_ci': mae_ci, 'rmse': rmse, 'rmse_ci': rmse_ci}

def evaluate(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad(), autocast(device_type='cuda', dtype=torch.float16):
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)
            preds.append(out.view(-1))
            trues.append(batch.y.view(-1))
    preds = torch.cat(preds)
    trues = torch.cat(trues)
    return torch.sqrt(torch.mean((preds - trues) ** 2)).item()

def evaluate_with_ci(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad(), autocast(device_type='cuda', dtype=torch.float16):
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)
            y_true = batch.y.view(-1).cpu().tolist()
            y_pred = out.view(-1).cpu().tolist()
            trues.extend(y_true)
            preds.extend(y_pred)
    return compute_metrics_with_ci(trues, preds)



In [7]:
loading = True
if loading:
    train_set = torch.load('/content/drive/MyDrive/lipophil_polyatomic_data/polyatomic_data_lipophil.pt', weights_only=False)
    test_set = torch.load('/content/drive/MyDrive/lipophil_polyatomic_data/polyatomic_test_data_lipophil.pt', weights_only=False)

In [8]:
import warnings
warnings.filterwarnings("ignore", message=".*An output with one or more elements was resized.*")
warnings.filterwarnings("ignore", message=".*FutureWarning:.*")
warnings.filterwarnings("ignore", category=UserWarning, module="torch.optim.lr_scheduler")

In [9]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=8, pin_memory=True)
test_loader  = DataLoader(test_set,  batch_size=128)

In [10]:
node_feat_dim = train_set[0].x.shape[1]
edge_feat_dim = train_set[0].edge_attr.shape[1]
graph_feat_dim = train_set[0].graph_feats.shape[0]

from torch_geometric.utils import degree

deg = torch.zeros(128, dtype=torch.long)

for data in train_loader:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    bc = torch.bincount(d, minlength=deg.size(0))
    if bc.size(0) > deg.size(0):
        new_deg = torch.zeros(bc.size(0), dtype=torch.long)
        new_deg[:deg.size(0)] = deg
        deg = new_deg
    deg += bc

model = PolyatomicNet(
    node_feat_dim=node_feat_dim,
    edge_feat_dim=edge_feat_dim,
    graph_feat_dim=graph_feat_dim,
    deg=deg,
)

input_dim = train_set[0].x.size(1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=10, factor=0.5, verbose=True
)
loss_fn   = nn.SmoothL1Loss(beta=0.5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [11]:
history = {}
N = 50
for epoch in range(1, N):
    tr_loss = train(model, train_loader, optimizer, loss_fn)
    metrics = evaluate_with_ci(model, test_loader)
    print(f"Epoch {epoch:02d} | Train Loss: {tr_loss:.4f} | "
          f"Test MAE: {metrics['mae']:.4f} (95% CI [{metrics['mae_ci'][0]:.4f}, {metrics['mae_ci'][1]:.4f}]) | "
          f"Test RMSE: {metrics['rmse']:.4f} (95% CI [{metrics['rmse_ci'][0]:.4f}, {metrics['rmse_ci'][1]:.4f}])")
    history[epoch] = metrics
    scheduler.step(metrics['mae'])

Epoch 01 | Train Loss: 0.5943 | Test MAE: 0.8046 (95% CI [0.7590, 0.8513]) | Test RMSE: 1.0218 (95% CI [0.9617, 1.0806])
Epoch 02 | Train Loss: 0.5771 | Test MAE: 0.8002 (95% CI [0.7539, 0.8467]) | Test RMSE: 1.0185 (95% CI [0.9580, 1.0792])
Epoch 03 | Train Loss: 0.5605 | Test MAE: 0.7535 (95% CI [0.7103, 0.7986]) | Test RMSE: 0.9599 (95% CI [0.9031, 1.0171])
Epoch 04 | Train Loss: 0.5315 | Test MAE: 0.7404 (95% CI [0.7016, 0.7805]) | Test RMSE: 0.9239 (95% CI [0.8766, 0.9719])
Epoch 05 | Train Loss: 0.5032 | Test MAE: 0.8859 (95% CI [0.8412, 0.9308]) | Test RMSE: 1.0662 (95% CI [1.0193, 1.1121])
Epoch 06 | Train Loss: 0.5014 | Test MAE: 0.7826 (95% CI [0.7362, 0.8297]) | Test RMSE: 1.0081 (95% CI [0.9490, 1.0657])
Epoch 07 | Train Loss: 0.4802 | Test MAE: 0.7119 (95% CI [0.6728, 0.7523]) | Test RMSE: 0.8873 (95% CI [0.8403, 0.9350])
Epoch 08 | Train Loss: 0.4727 | Test MAE: 0.7104 (95% CI [0.6703, 0.7509]) | Test RMSE: 0.8859 (95% CI [0.8378, 0.9337])
Epoch 09 | Train Loss: 0.4673 | 

In [12]:
final = history[N-1]
print("*"*20)
print(f"Test MAE: {final['mae']:.4f} (95% CI [{final['mae_ci'][0]:.4f}, {final['mae_ci'][1]:.4f}])")
print(f"Test RMSE: {final['rmse']:.4f} (95% CI [{final['rmse_ci'][0]:.4f}, {final['rmse_ci'][1]:.4f}])")
print("*"*20)

********************
Test MAE: 0.5366 (95% CI [0.5050, 0.5700])
Test RMSE: 0.7033 (95% CI [0.6608, 0.7471])
********************
